In [2]:
import networkx as nx

# 1. Lê o arquivo de entrada
with open('entrada-processada.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 2. Cria o grafo direcionado
G = nx.DiGraph()

# 3. Processa cada linha
for line in lines:
    if line.startswith('("entity'):
        # Exemplo: ("entity"{{tuple_delimiter}}LARGE LANGUAGE MODELS{{tuple_delimiter}}large language model{{tuple_delimiter}}Large language models (LLMs) are advanced AI models designed...
        parts = line.split('{{tuple_delimiter}}')
        entity_name = parts[1].strip()
        entity_type = parts[2].strip()
        entity_desc = parts[3].split(')')[0].strip()  # até fechar o parênteses

        # Adiciona como nó com atributos
        G.add_node(entity_name, type=entity_type, description=entity_desc)

    elif line.startswith('("relationship'):
        # Exemplo: ("relationship"{{tuple_delimiter}}LARGE LANGUAGE MODELS{{tuple_delimiter}}DATASETS{{tuple_delimiter}}Large language models are trained using large datasets{{tuple_delimiter}}8
        parts = line.split('{{tuple_delimiter}}')
        src = parts[1].strip()
        tgt = parts[2].strip()
        relation_desc = parts[3].strip()

        # Adiciona como aresta com descrição
        G.add_edge(src, tgt, description=relation_desc)

print(f"Grafo criado com {G.number_of_nodes()} nós e {G.number_of_edges()} arestas.")

# (Opcional) 4. Exemplo: listar as relações
for u, v, data in G.edges(data=True):
    print(f"{u} -> {v} : {data['description']}")


Grafo criado com 24 nós e 10 arestas.
LARGE LANGUAGE MODELS -> DATASETS : Large language models are trained using large datasets that provide the diverse text data necessary for learning
LARGE LANGUAGE MODELS -> DIFFERENTIAL PRIVACY : Differential privacy is applied to large language models to protect sensitive information in training data
FEDERATED LEARNING -> HEALTHCARE : Federated learning is used in healthcare to train models on patient data without compromising privacy
ADVERSARIAL TRAINING -> SECURITY MEASURES : Adversarial training is a security measure used to enhance the robustness of models against attacks
LEARNING RATE -> ADAGRAD : AdaGrad adjusts the learning rate dynamically during model training
LEARNING RATE -> RMSPROP : RMSprop is another optimizer that modifies the learning rate to improve training stability
ADAPTER ARCHITECTURE -> LORA : LoRA is an alternative adapter architecture for efficient fine-tuning of large models
DATA PREPROCESSING -> DATA IMBALANCE : Data pre

In [3]:
from difflib import get_close_matches

def buscar_no_grafo(pergunta, top_k=3):
    # 1. Lista todos os nós
    todos_nos = list(G.nodes)
    
    # 2. Usa fuzzy match para encontrar nós mais similares à pergunta
    matches = get_close_matches(pergunta.upper(), todos_nos, n=top_k, cutoff=0.1)
    
    contexto = []
    for match in matches:
        node_data = G.nodes[match]
        contexto.append(f"Entidade: {match} ({node_data['type']})\nDescrição: {node_data['description']}")
        
        # 3. Pega relações de saída
        for _, target, edge_data in G.out_edges(match, data=True):
            contexto.append(f"Relacionamento: {match} -> {target}\nDescrição: {edge_data['description']}")
        
        # 4. Pega relações de entrada
        for source, _, edge_data in G.in_edges(match, data=True):
            contexto.append(f"Relacionamento: {source} -> {match}\nDescrição: {edge_data['description']}")
    
    return "\n\n".join(contexto)


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ✅ Detecta se há GPU (CUDA) disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔎 Usando dispositivo: {device}")

# ✅ Passo 2: Carregar TinyLlama (sem device_map)
print("🔄 Carregando modelo TinyLlama...")
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)
print("✅ Modelo TinyLlama carregado!")


🔎 Usando dispositivo: cpu
🔄 Carregando modelo TinyLlama...
✅ Modelo TinyLlama carregado!


In [ ]:
def responder_graphrag(pergunta_usuario):
    # 🔍 Passo 1: Buscar contexto no grafo
    contexto = buscar_no_grafo(pergunta_usuario)
    
    # 🧠 Passo 2: Montar prompt estilo RAG
    prompt = f"""
        Você é um assistente especializado em inteligência artificial e privacidade.
        
        Use **exclusivamente** o contexto abaixo para responder à pergunta do usuário.
        
        Contexto:
        {contexto}
        
        Pergunta: {pergunta_usuario}
        
        Resposta:
    """
    # Tokeniza
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Gera resposta
    outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.7, do_sample=True)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Só extrai a parte da resposta (remove o prompt inicial)
    resposta_final = resposta.split("Resposta:")[-1].strip()
    return resposta_final

# ===========================
# 5. Teste interativo!
# ===========================

while True:
    pergunta = input("\n❓ Sua pergunta (ou 'sair'): ")
    if pergunta.lower() == "sair":
        break
    resposta = responder_graphrag(pergunta)
    print("\n🤖 Resposta do GraphRAG:\n", resposta)


❓ Sua pergunta (ou 'sair'):  What is data privacy?
